## Etapas

1. Criação de features
2. Tratamento de outliers
3. Normalização dos dados
4. Ajustes nos dados
5. Pipeline
6. Separação em treino/teste
7. Treinando modelo

In [ ]:
# @title Importação de bibliotecas

# Dataset
import pandas as pd
import numpy as np
import requests
from pathlib import Path

# Criação de features
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

# Tratamento de outliers
from sklearn.base import BaseEstimator, TransformerMixin

# Ajustes nos dados
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

# Treinamento do modelo 
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate, TimeSeriesSplit

In [ ]:
# @title Carregamento do dataset

permalink = "https://github.com/mfigueireddo/ciencia-de-dados/blob/ba579573c5b8a9246ca04f7da29bc2c74c8b362c/datasets/pre-pipeline_wildfires.parquet"
raw_url = permalink.replace("/blob/", "/raw/")

dest = Path("/content/raw_wildfires.parquet")

with requests.get(raw_url, stream=True) as r:
    r.raise_for_status()
    with open(dest, "wb") as f:
        for chunk in r.iter_content(chunk_size=1024 * 1024):
            if chunk:
                f.write(chunk)

# Leitura do arquivo .csv
wildfires = pd.read_parquet(dest, engine="pyarrow")

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/QuickAccess/pre-pipeline_wildfires.csv'

## 1. Criação de features

In [ ]:
# @title Criação de features

class TemporalGeoFeatureEngineer(BaseEstimator, TransformerMixin):
    """
    Gera:
      - estacao_ano_num  (0=Verão, 1=Outono, 2=Inverno, 3=Primavera)
      - regiao_incendio  (cluster DBSCAN atribuído por vizinho-core mais próximo; -1 = ruído/sem cluster)
      - soma_precipitacao_14d (rolling por grupo)
      - media_temp_max_7d     (rolling por grupo)

    Observação: as primeiras linhas de cada grupo podem virar NaN pelas janelas;
    trate isso depois na própria Pipeline (ex.: SimpleImputer).
    """
    def __init__(self,
                 date_col='data',
                 group_col='fire_id',
                 lat_col='latitude',
                 lon_col='longitude',
                 precip_col='precipitacao',
                 tmax_col='temp_max',
                 window_precip=14,
                 window_tmax=7,
                 eps_km=1.0,
                 min_samples=5):
        self.date_col = date_col
        self.group_col = group_col
        self.lat_col = lat_col
        self.lon_col = lon_col
        self.precip_col = precip_col
        self.tmax_col = tmax_col
        self.window_precip = window_precip
        self.window_tmax = window_tmax
        self.eps_km = eps_km
        self.min_samples = min_samples

        # Objetos aprendidos no fit
        self._dbscan_ = None
        self._nn_core_ = None
        self._core_labels_ = None
        self._eps_rad_ = None

    @staticmethod
    def _to_radians(latlon):
        return np.radians(latlon.astype(float))

    @staticmethod
    def _month_to_season_south(month):
        # Estações (aprox.) para hemisfério sul:
        # Verão:   Dez(12)-Fev(2)  -> 0
        # Outono:  Mar(3)-Mai(5)   -> 1
        # Inverno: Jun(6)-Ago(8)   -> 2
        # Primavera: Set(9)-Nov(11)-> 3
        if month in (12, 1, 2):   return 0
        if month in (3, 4, 5):    return 1
        if month in (6, 7, 8):    return 2
        return 3  # 9,10,11

    def fit(self, X, y=None):
        X = X.copy()
        missing_cols = [c for c in [self.lat_col, self.lon_col] if c not in X.columns]
        if missing_cols:
            # Sem lat/lon não há cluster geoespacial — seguimos sem DBSCAN
            self._dbscan_ = None
            self._nn_core_ = None
            self._core_labels_ = None
            self._eps_rad_ = None
            return self

        # DBSCAN em coordenadas (haversine espera radianos)
        latlon = X[[self.lat_col, self.lon_col]].to_numpy()
        latlon_rad = self._to_radians(latlon)

        # converter eps de km para radianos (Raio médio Terra ~ 6371 km)
        self._eps_rad_ = self.eps_km / 6371.0

        db = DBSCAN(eps=self._eps_rad_, min_samples=self.min_samples, metric='haversine')
        db.fit(latlon_rad)
        self._dbscan_ = db

        # Treina um NN apenas nos pontos-core para atribuição em transform()
        core_mask = np.zeros_like(db.labels_, dtype=bool)
        if hasattr(db, 'core_sample_indices_') and len(db.core_sample_indices_) > 0:
            core_mask[db.core_sample_indices_] = True
            core_points = latlon_rad[core_mask]
            core_labels = db.labels_[core_mask]

            if len(core_points) > 0:
                nn = NearestNeighbors(n_neighbors=1, metric='haversine')
                nn.fit(core_points)
                self._nn_core_ = nn
                self._core_labels_ = core_labels
            else:
                self._nn_core_ = None
                self._core_labels_ = None
        else:
            self._nn_core_ = None
            self._core_labels_ = None

        return self

    def _assign_dbscan_labels(self, X):
        # Se não tivemos lat/lon ou DBSCAN treinado, devolve NaN
        if self._nn_core_ is None or self._core_labels_ is None or self._eps_rad_ is None:
            return pd.Series([-1] * len(X), index=X.index, dtype='int64')

        latlon = X[[self.lat_col, self.lon_col]].to_numpy()
        latlon_rad = self._to_radians(latlon)

        # Atribui rótulo do core mais próximo, desde que dentro do raio eps
        distances, indices = self._nn_core_.kneighbors(latlon_rad, n_neighbors=1, return_distance=True)
        distances = distances.reshape(-1)
        indices = indices.reshape(-1)

        labels = np.full(len(X), -1, dtype='int64')
        within = distances <= self._eps_rad_
        labels[within] = self._core_labels_[indices[within]]
        return pd.Series(labels, index=X.index, dtype='int64')

    def _add_temporal_rollings(self, df):
        # Ordena por grupo e tempo para garantir rolling correto
        if self.group_col in df.columns and self.date_col in df.columns:
            df = df.sort_values([self.group_col, self.date_col])
        else:
            # Se faltar algo, só ordena por data se existir
            if self.date_col in df.columns:
                df = df.sort_values(self.date_col)

        # Rolling de precipitação (soma 14d)
        if self.precip_col in df.columns:
            df['soma_precipitacao_14d'] = (
                df.groupby(self.group_col, dropna=False)[self.precip_col]
                  .rolling(self.window_precip, min_periods=1)
                  .sum()
                  .reset_index(level=0, drop=True)
            )
        else:
            df['soma_precipitacao_14d'] = np.nan

        # Rolling de temp máxima (média 7d)
        if self.tmax_col in df.columns:
            df['media_temp_max_7d'] = (
                df.groupby(self.group_col, dropna=False)[self.tmax_col]
                  .rolling(self.window_tmax, min_periods=1)
                  .mean()
                  .reset_index(level=0, drop=True)
            )
        else:
            df['media_temp_max_7d'] = np.nan

        return df

    def transform(self, X):
        # Trabalha em DataFrame para manter nomes/índices
        df = pd.DataFrame(X).copy()

        # 1) Estação do ano (numérica, 0..3)
        if self.date_col in df.columns:
            # Garante dtype datetime
            df[self.date_col] = pd.to_datetime(df[self.date_col], errors='coerce')
            estacao = df[self.date_col].dt.month.map(self._month_to_season_south).astype('Int64')
            df['estacao_ano_num'] = estacao.astype('float').astype('Int64')  # evita problemas de NaN -> imputar depois
            df['estacao_ano_num'] = df['estacao_ano_num'].astype('float')
        else:
            df['estacao_ano_num'] = np.nan

        # 2) Região por DBSCAN (atribuída por NN para dados novos)
        if all(c in df.columns for c in [self.lat_col, self.lon_col]):
            df['regiao_incendio'] = self._assign_dbscan_labels(df).astype('int64')
        else:
            df['regiao_incendio'] = -1

        # 3) Rollings temporais por grupo
        df = self._add_temporal_rollings(df)

        return df


## 2. Tratamento de outliers

**Transformação Logarítmica**
- Aplica log(x) ou log(x+constante) para valores positivos
- Muito eficaz para dados com distribuição assimétrica positiva
- Comprime valores grandes e expande valores pequenos
- Fórmula: X_log = log(X + c), onde c evita log(0)

**Transformação Raiz Quadrada**
- Menos drástica que a transformação logarítmica
- Útil para dados de contagem e variáveis positivamente assimétricas
- Fórmula: X_sqrt = sqrt(X)

**Winsorização (Capping/Clipping)**

A **Winsorização** é uma técnica de tratamento de outliers que **limita valores extremos** sem removê-los completamente. Em vez de excluir outliers, substituímos os valores extremos pelos valores de percentis específicos.

**Como funciona:**
- Define-se limites baseados em percentis (ex: 5º e 95º percentil)
- Valores abaixo do limite inferior são substituídos pelo valor do limite inferior
- Valores acima do limite superior são substituídos pelo valor do limite superior

| Variável                                 | Melhor método         | Justificativa (1 linha)                                                                                   |
| :--------------------------------------- | :-------------------- | :-------------------------------------------------------------------------------------------------------- |
| **precipitacao**                         | **Log(x + 1)**        | Reduziu fortemente a assimetria (7.87 → 2.59) e manteve os limites IQR estáveis — ideal para cauda longa. |
| **umidade_relativa_max**                 | **Winsorizar 5%-5%**  | Cortou outliers (23 → 0) e melhorou levemente a simetria; log/sqrt inflaram valores.                      |
| **umidade_relativa_min**                 | **Winsorizar 5%-5%**  | Assimetria e outliers foram totalmente corrigidos (1155 → 0).                                             |
| **umidade_especifica**                   | **Sqrt**              | Melhor simetria (0.89 → 0.16) e forte redução de outliers (6967 → 2102).                                  |
| **radiacao_solar**                       | **Sem transformação** | Já simétrica e sem outliers; log piorou, winsor apenas repete.                                            |
| **temperatura_min**                      | **Winsorizar 5%-5%**  | Remoção completa de outliers (5066 → 0) e leve ganho de simetria.                                         |
| **temperatura_max**                      | **Winsorizar 5%-5%**  | Mesmo comportamento de `temperatura_min`.                                                                 |
| **velocidade_vento**                     | **Log(x + 1)**        | Reduziu assimetria (1.23 → 0.19) e outliers (8723 → 1128) sem eliminar extremos reais.                    |
| **indice_queima**                        | **Winsorizar 5%-5%**  | Log e sqrt aumentaram outliers via IQR; winsor eliminou-os com mínima distorção.                          |
| **umidade_combustivel_morto_100_horas**  | **Sqrt**              | Forte queda de outliers (44 → 9) e leve suavização de forma.                                              |
| **umidade_combustivel_morto_1000_horas** | **Sqrt**              | Reduziu outliers (373 → 4) mantendo distribuição coerente.                                                |
| **componente_energia_lancada**           | **Sem transformação** | Já equilibrada; transformações criam falsos outliers.                                                     |
| **evapotranspiracao_real**               | **Sqrt**              | Melhorou drasticamente a simetria (0.71 → -0.00) e reduziu outliers (3292 → 153).                         |
| **evapotranspiracao_potencial**          | **Log(x + 1)**        | Skew caiu (0.53 → -0.36) e outliers zeraram (679 → 0).                                                    |
| **deficit_pressao_vapor**                | **Log(x + 1)**        | Alta cauda direita suavizada (1.46 → 0.52) e outliers despencaram (14758 → 672).                          |


In [ ]:
# @title 2. Tratamento de outliers

log_cols = [
    "precipitacao",
    "velocidade_vento",
    "evapotranspiracao_potencial",
    "deficit_pressao_vapor",
]
sqrt_cols = [
    "umidade_especifica",
    "umidade_combustivel_morto_100_horas",
    "umidade_combustivel_morto_1000_horas",
    "evapotranspiracao_real",
]
winsor_cols = [
    "umidade_relativa_max",
    "umidade_relativa_min",
    "temperatura_min",
    "temperatura_max",
    "indice_queima",
]

class LogSqrtWinsorizer(BaseEstimator, TransformerMixin):

    def __init__(self, log_cols=None, sqrt_cols=None, winsor_cols=None, winsor_limits=(0.05, 0.05)):
        self.log_cols = log_cols or []
        self.sqrt_cols = sqrt_cols or []
        self.winsor_cols = winsor_cols or []
        self.winsor_limits = winsor_limits

    def fit(self, X, y=None):
        Xdf = X if isinstance(X, pd.DataFrame) else pd.DataFrame(X)

        self.log_offset_ = {}
        for c in self.log_cols:
            if c in Xdf.columns:
                s = pd.to_numeric(Xdf[c], errors="coerce")
                m = s.min()
                self.log_offset_[c] = (abs(m) + 1) if pd.notna(m) and m <= 0 else 1.0

        self.sqrt_offset_ = {}
        for c in self.sqrt_cols:
            if c in Xdf.columns:
                s = pd.to_numeric(Xdf[c], errors="coerce")
                m = s.min()
                self.sqrt_offset_[c] = (abs(m) + 0.01) if pd.notna(m) and m < 0 else 0.0

        present_winsor = [c for c in self.winsor_cols if c in Xdf.columns]
        if present_winsor:
            q_low, q_high = self.winsor_limits
            self.low_  = pd.to_numeric(Xdf[present_winsor], errors="coerce").quantile(q_low)
            self.high_ = pd.to_numeric(Xdf[present_winsor], errors="coerce").quantile(1 - q_high)
        else:
            self.low_ = pd.Series(dtype=float)
            self.high_ = pd.Series(dtype=float)

        return self

    def transform(self, X):
        Xdf = X.copy() if isinstance(X, pd.DataFrame) else pd.DataFrame(X).copy()

        # LOG
        for c, off in self.log_offset_.items():
            if c in Xdf.columns:
                s = pd.to_numeric(Xdf[c], errors="coerce")
                Xdf[c] = np.log(s + off)

        # SQRT
        for c, off in self.sqrt_offset_.items():
            if c in Xdf.columns:
                s = pd.to_numeric(Xdf[c], errors="coerce")
                Xdf[c] = np.sqrt(s + off)

        # Winsorização
        for c in self.low_.index:
            if c in Xdf.columns:
                s = pd.to_numeric(Xdf[c], errors="coerce")
                Xdf[c] = s.clip(lower=self.low_[c], upper=self.high_[c])

        return Xdf

## 3. Normalização dos dados

In [ ]:
# @title 3. Normalização dos dados

## 4. Ajustes nos dados

In [ ]:
# @title Ajustes nos dados

# Colunas categóricas criadas no passo de features
_cat_cols = ['estacao_ano_num', 'regiao_incendio']

def _numeric_selector(X):
    """
    Seleciona colunas numéricas, exceto as categóricas codificadas numericamente.
    (Funciona porque o ColumnTransformer aceita callables como seletores.)
    """
    num = X.select_dtypes(include=[np.number]).columns.tolist()
    return [c for c in num if c not in _cat_cols]

_numeric_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler',  StandardScaler()),
])

_categorical_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe',     OneHotEncoder(handle_unknown='ignore', sparse=False)),
])

## 5. Pipeline

In [ ]:
# @title Pipeline

preprocess = Pipeline(steps=[
    # Criação de features
    ('features', TemporalGeoFeatureEngineer(
        date_col='data',
        group_col='fire_id',
        lat_col='latitude',
        lon_col='longitude',
        precip_col='precipitacao', 
        tmax_col='temp_max',
        window_precip=14,
        window_tmax=7,
        eps_km=0.96,   # ~0.015 rad * 6371 ≈ 95.6km (compatível com seu eps anterior)
        min_samples=5
    )),
    # Tratamento de outliers
    ("outliers", LogSqrtWinsorizer(
        log_cols=log_cols,
        sqrt_cols=sqrt_cols,
        winsor_cols=winsor_cols,
        winsor_limits=(0.05, 0.05),
    )),
    # Normalização dos dados
    #("scaler",   FunctionTransformer(scale_features,  validate=False)),
    # Separação em treino/teste
])


## 6. Separação em treino/teste

### Time Series Cross Validation

A Time Series Cross Validation é uma técnica especializada para validar modelos quando os dados possuem ordem cronológica. Diferente das técnicas tradicionais, ela respeita a estrutura temporal dos dados.

In [ ]:
# @title 4. Separação em treino/teste

def group_time_series_cv(
    df: pd.DataFrame,
    time_col: str,
    group_col: str,
    *,
    n_splits: int = 5,
    test_groups_size: int = 1,
    gap_groups: int = 0,
    expanding: bool = True,
    min_train_groups: int | None = None,
    step_groups: int | None = None,
):
    """
    Gera folds (train_idx, test_idx) respeitando ordem temporal por grupo,
    embargo em nível de grupo e exclusão mútua treino/teste por grupo.
    """
    # 1) Ordena grupos pelo primeiro timestamp
    first_time = (
        df[[group_col, time_col]]
        .dropna(subset=[time_col])
        .groupby(group_col)[time_col]
        .min()
        .sort_values()
    )
    ordered_groups = first_time.index.to_numpy()
    n_groups = len(ordered_groups)

    if step_groups is None:
        step_groups = test_groups_size
    if min_train_groups is None:
        # mínimo sensato: pelo menos o tamanho do primeiro teste
        min_train_groups = max(1, test_groups_size)

    # Âncora: último grupo incluso no treino
    # Precisamos garantir espaço para gap + teste à frente
    max_anchor = n_groups - gap_groups - test_groups_size
    if max_anchor <= min_train_groups:
        return  # não há splits possíveis

    splits = 0
    anchor = min_train_groups
    while anchor <= max_anchor and splits < n_splits:
        if expanding:
            train_groups = ordered_groups[:anchor]
        else:
            start = max(0, anchor - min_train_groups)
            train_groups = ordered_groups[start:anchor]

        test_start = anchor + gap_groups
        test_end = test_start + test_groups_size
        test_groups = ordered_groups[test_start:test_end]

        train_idx = df.index[df[group_col].isin(train_groups)].to_numpy()
        test_idx  = df.index[df[group_col].isin(test_groups)].to_numpy()

        if train_idx.size and test_idx.size:
            yield (train_idx, test_idx)
            splits += 1

        anchor += step_groups


In [ ]:
# @title Executando separação

cv_splits = list(group_time_series_cv(
    df=wildfires,
    time_col='data',
    group_col='fire_id',
    n_splits=5,
    test_groups_size=1,
    gap_groups=0,
    expanding=True,
))

## 7. Treinando modelo

In [ ]:
# Separar X e y (mantém 'data' e 'fire_id' porque o preprocess usa essas colunas)
X = wildfires.drop(columns=["houve_incendio"])
y = wildfires["houve_incendio"].astype(int)

resultados = []

modelos = {
    "Dummy (mais frequente)": DummyClassifier(strategy="most_frequent"),
    "Regressão Logística": LogisticRegression(max_iter=1000),
    "Árvore de Decisão": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=50, n_jobs=-1),
    "Naive Bayes": GaussianNB(),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "SVM (linear)": SVC(kernel='linear')  # Deixe por último, ele n ta indo (apagar quando for rodar, depois vou tentar arrumar)
}

# Usar os folds temporais por grupo já definidos anteriormente
# (cv_splits veio da célula "Pipeline" com group_time_series_cv(...))
cv = cv_splits

scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score, zero_division=0),
    'recall': make_scorer(recall_score, zero_division=0),
    'f1': make_scorer(f1_score, zero_division=0),
    'roc_auc': 'roc_auc'  # deixa o sklearn decidir entre predict_proba/decision_function
}

for nome, modelo in modelos.items():
    print(f"Treinando modelo: {nome}...")
    try:
        pipeline = Pipeline([
            ("preprocess", preprocess),  # usa a nossa pipeline pronta
            ("clf", modelo)
        ])

        scores = cross_validate(pipeline, X, y, cv=cv, scoring=scoring)

        resultados.append({
            "Modelo": nome,
            "Accuracy": np.mean(scores['test_accuracy']),
            "Precision": np.mean(scores['test_precision']),
            "Recall": np.mean(scores['test_recall']),
            "F1-score": np.mean(scores['test_f1']),
            "ROC AUC": np.mean(scores['test_roc_auc']),
        })

        print(f" Modelo {nome} treinado com sucesso.\n")

    except Exception as e:
        print(f" Erro ao rodar o modelo {nome}: {e}\n")

# Mostrar resultados
df_resultados = pd.DataFrame(resultados).sort_values("F1-score", ascending=False)
df_resultados
